## Задание теме «Коллаборативная фильтрация»

ПАКЕТ SURPRISE

 - используйте данные MovieLens 1M   
 - можно использовать любые модели из пакета   
 - получите RMSE на тестовом сете 0.87 и ниже   

In [41]:
from surprise import KNNWithMeans, KNNBasic, KNNWithMeans, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split, GridSearchCV 

import pandas as pd
import numpy as np


In [10]:
movies = pd.read_csv('movies.dat',sep='::',names=['movieId','title','genres'])
ratings = pd.read_csv('ratings.csv',sep='::',names=['userId','movieId','rating','timestamp'])

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [31]:
ratings.shape[0]

1000209

В выборке ~ 1 миллион оценок

In [22]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [23]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [26]:
reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [27]:
trainset, testset = train_test_split(data, test_size=.15)

In [33]:
model_list = [KNNWithMeans(), KNNBasic(), KNNWithMeans(), KNNBaseline()]

In [35]:

for model in model_list:    
    model.fit(trainset)
    test_pred = model.test(testset)
    accuracy.rmse(test_pred, verbose=True)
    print(accuracy.rmse(test_pred, verbose=True))
    

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9284
RMSE: 0.9284
0.9284034408447813
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9183
RMSE: 0.9183
0.9183210876569908
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9284
RMSE: 0.9284
0.9284034408447813
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8927
RMSE: 0.8927
0.8926531611068637


Самое маленькое RMSE на KNNBaseline(). Попробуем улучшить точность

In [47]:
param_grid = {
            'k': np.arange(45, 60, 5),
            'sim_options': {'name': ['pearson_baseline'], 
                            'user_based': [False]}, 
            'min_k': np.arange(1, 3)
            }

In [48]:
grid_search = GridSearchCV(KNNBaseline, param_grid, measures=['rmse'], cv=5)

In [49]:
grid_search.fit(data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [50]:
grid_search.best_params['rmse']

{'k': 45,
 'sim_options': {'name': 'pearson_baseline', 'user_based': False},
 'min_k': 2}

In [55]:
model = KNNBaseline(k = 45, name='pearson_baseline', user_based=False, min_k=2)
model.fit(trainset)
predictions = best_alg.test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8557


0.8557401100096524

In [56]:
model.predict(uid=2, iid='Fight Club (1999)')

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.093026254301611, details={'actual_k': 45, 'was_impossible': False})